### Импорт библиотек


In [ ]:
%pylab inline
import numpy as np
import pandas as pd
import pylab as plt
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard


from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

import seaborn as sns
sns.set(style="darkgrid")
from IPython.display import Markdown, display
from sklearn.linear_model import Ridge

import ppscore as pps

def printmd(string):
    display(Markdown(string))

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999


RND_SEED=1987

### Функции для оценки

In [ ]:
#функция для отбора нейросетей
def ann_model(Xtrain, ytrain, Xtest, ytest):
    min_mse = 1  
    for n_neurons in range(1,4):
        for i in [1e-2,1e-3,1e-4]:
            model = Sequential()
            model.add(tf.keras.Input(shape=(1,)))
            model.add(Dense(n_neurons, activation='linear'))
            model.add(Dense(1, activation='linear'))
            opt = optimizers.Adam(lr = i)
            model.compile(loss = 'mean_squared_error', optimizer = opt)
            history = model.fit(Xtrain, ytrain, validation_data = (Xtest, ytest), epochs = 50, verbose = 0)

            # Plot training & validation loss values

            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            title_text = 'Feature_name: ' + str(Xtrain)[2:8] + '\n' + 'Model parameters:' + str(n_neurons) + ' neurons, ' + str(i) + ' learning rate'
            plt.title(title_text)
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Test'], loc='upper left')
            plt.show()
            print('Ошибка на тестовом сете: ', history.history['val_loss'][-1])

            current_mse = history.history['val_loss'][-1]
            if current_mse < min_mse:
                min_mse = current_mse
                best_model = model

    return best_model, min_mse

#составление массива с прогнозами
def fill_predictions(predictions, currency_pair, model):
    #заполняем раздел в файле прогнозов
    #pred = np.reshape(model.predict([means_without_na[currency_pair][1]]), -1)
    predictions[currency_pair][0] = means_without_na[currency_pair][0]

    for i in range(1, 13):
        pred = np.reshape(ann.predict([float(predictions[currency_pair][i-1])]), -1)
        predictions[currency_pair][i] = pred[0]

    return predictions

### Предобработка данных ДВК

In [ ]:
#read file for DVK forecast

df = pd.read_excel('DVK.xlsx', sheet_name='sheet0', header=None)
print(df.shape)

#cleaning dataframe
df.drop([0,1,2,3,4,5], axis = 0, inplace = True)
df.reset_index(inplace = True, drop = True)
#creating names for cols
df.columns = ['date','eurusd','eurchf','gbpusd']

# calculate monthly averages
count = 0
index = 0
sumeur = 0
sumchf = 0
sumgbp = 0
eurusd_mean = []
chf_mean = []
gbp_mean = []

for i in range(df.shape[0]-1):
    if df['date'].iloc[index].month == df['date'].iloc[index+1].month:
        sumeur = sumeur + df['eurusd'].iloc[index]
        sumchf = sumchf + df['eurchf'].iloc[index]
        sumgbp = sumgbp + df['gbpusd'].iloc[index]
        
        count = count + 1
        index = index + 1
    else:
        sumeur = sumeur + df['eurusd'].iloc[index]
        sumchf = sumchf + df['eurchf'].iloc[index]
        sumgbp = sumgbp + df['gbpusd'].iloc[index]
        
        count = count + 1
                
        mean_eur = sumeur/count
        eurusd_mean.append(mean_eur)
        
        mean_chf = sumchf/count
        chf_mean.append(mean_chf)
        
        mean_gbp = sumgbp/count
        gbp_mean.append(mean_gbp)
        
        
        count = 0
        sumchf = 0
        sumgbp = 0
        sumeur = 0
        index = index + 1

#forming dataframe from monthly means

means = pd.DataFrame(
    {'eurusd':eurusd_mean,
     'eurchf':chf_mean,
     'gbpusd':gbp_mean
    })       
means_without_na = means[0:255]

# создаем массив лагов для каждой валютной пары
X_eurusd = means_without_na['eurusd'][1:]
X_eurchf = means_without_na['eurchf'][1:]
X_gbpusd = means_without_na['gbpusd'][1:]

#создаем массив целевых переменных
y_eurusd = means_without_na['eurusd'][0:254]
y_eurchf = means_without_na['eurchf'][0:254]
y_gbpusd = means_without_na['gbpusd'][0:254]

#Сделаем разбивку на трейн и тест
breakpoint = 30
X_eurusd_train = X_eurusd[breakpoint:]
X_eurusd_test = X_eurusd[0:breakpoint]
X_eurchf_train = X_eurchf[breakpoint:]
X_eurchf_test = X_eurchf[0:breakpoint]
X_gbpusd_train = X_gbpusd[breakpoint:]
X_gbpusd_test = X_gbpusd[0:breakpoint]

y_eurusd_train = y_eurusd[breakpoint:]
y_eurusd_test = y_eurusd[0:breakpoint]
y_eurchf_train = y_eurchf[breakpoint:]
y_eurchf_test = y_eurchf[0:breakpoint]
y_gbpusd_train = y_gbpusd[breakpoint:]
y_gbpusd_test = y_gbpusd[0:breakpoint]

#массив предиктов
predictions = pd.DataFrame(columns = ['fed', 'ecb', 'eurusd', 'eurchf', 'gbpusd', 'bunds2', 'bunds5', 'bunds10', 'ust2', 'ust5', 'ust10', 'eu_libor3', 'us_libor3'], index=['m+0', 'm+1', 'm+2', 'm+3', 'm+4', 'm+5', 'm+6', 'm+7', 'm+8', 'm+9', 'm+10', 'm+11', 'm+12'])

    
    


### Обучение и прогноз

In [ ]:
# прогоняем модель на всех трех валютных парах
# формируем массив прогнозов
best_model, min_mse = ann_model(X_eurusd_train, y_eurusd_train,X_eurusd_test, y_eurusd_test)
predictions = fill_predictions(predictions, 'eurusd', best_model)

best_model, min_mse = ann_model(X_gbpusd_train, y_gbpusd_train,X_gbpusd_test, y_gbpusd_test)
predictions = fill_predictions(predictions, 'gbpusd', best_model)

best_model, min_mse = ann_model(X_eurchf_train, y_eurchf_train,X_eurchf_test, y_eurchf_test)
predictions = fill_predictions(predictions, 'eurchf', best_model)

#транспонируем массив прогнозов для формата краткосрочного прогноза
predictions_vertical = predictions.T
predictions_vertical